<a href="https://colab.research.google.com/github/weasel-codes/anomaly-detection-iot/blob/main/anomal_detection_iot_preprocess1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

> Performing basic file split using wireshark

In [1]:
!apt-get install tshark

Reading package lists... Done
Building dependency tree       
Reading state information... Done
tshark is already the newest version (2.6.10-1~ubuntu18.04.0).
0 upgraded, 0 newly installed, 0 to remove and 30 not upgraded.


In [2]:
import os
import pandas as pd
import numpy as np
import glob

In [3]:
base_url = '/content/drive/MyDrive/ML/anomaly/'

# Import Dataset Description

In [4]:
file = pd.read_excel(base_url + 'dataset_description.xlsx')
desc_arr = file.to_numpy()
desc_arr.shape

(59, 5)

In [5]:
desc_arr[54:]

array([[nan, nan, 'Mirai Botnet', 'Telnet Bruteforce',
        'tcp.dstport==23 and ip.src==192.168.0.13'],
       [41.0, 'mirai-hostbruteforce-4-dec.pcap', 'Mirai Botnet',
        'Host Discovery',
        'arp.dst.proto_ipv4 == 192.168.0.0/24 and arp.src.proto_ipv4 == 192.168.0.24 and eth.dst == ff:ff:ff:ff:ff:ff'],
       [nan, nan, 'Mirai Botnet', 'Telnet Bruteforce',
        'tcp.dstport==23 and ip.src==192.168.0.24'],
       [42.0, 'mirai-hostbruteforce-5-dec.pcap', 'Mirai Botnet',
        'Host Discovery',
        'arp.dst.proto_ipv4 == 192.168.0.0/24 and arp.src.proto_ipv4 == 192.168.0.13 and eth.dst == ff:ff:ff:ff:ff:ff'],
       [nan, nan, 'Mirai Botnet', 'Telnet Bruteforce',
        'tcp.dstport==23 and ip.src==192.168.0.13']], dtype=object)

In [6]:
tot_file = desc_arr.shape[0]
for i in range(tot_file) :
    desc_arr[i][0] = i+1
    if(isinstance(desc_arr[i][1], float)) :
      desc_arr[i][1] = desc_arr[i-1][1]

desc_arr[54:]

array([[55, 'mirai-hostbruteforce-3-dec.pcap', 'Mirai Botnet',
        'Telnet Bruteforce', 'tcp.dstport==23 and ip.src==192.168.0.13'],
       [56, 'mirai-hostbruteforce-4-dec.pcap', 'Mirai Botnet',
        'Host Discovery',
        'arp.dst.proto_ipv4 == 192.168.0.0/24 and arp.src.proto_ipv4 == 192.168.0.24 and eth.dst == ff:ff:ff:ff:ff:ff'],
       [57, 'mirai-hostbruteforce-4-dec.pcap', 'Mirai Botnet',
        'Telnet Bruteforce', 'tcp.dstport==23 and ip.src==192.168.0.24'],
       [58, 'mirai-hostbruteforce-5-dec.pcap', 'Mirai Botnet',
        'Host Discovery',
        'arp.dst.proto_ipv4 == 192.168.0.0/24 and arp.src.proto_ipv4 == 192.168.0.13 and eth.dst == ff:ff:ff:ff:ff:ff'],
       [59, 'mirai-hostbruteforce-5-dec.pcap', 'Mirai Botnet',
        'Telnet Bruteforce', 'tcp.dstport==23 and ip.src==192.168.0.13']],
      dtype=object)

In [7]:
cpac_file_loc = 'packet/'
attack_key = 'attack/'
all_key = 'csv/'

In [8]:
# For benign
command = 'tshark -r ' + base_url + cpac_file_loc + desc_arr[0][1] + ' -T fields -e frame.number -e frame.time_relative -e frame.len -e data.len -e ip.src -e ip.dst -e frame.protocols -E header=y -E separator=, -E quote=d -E occurrence=f > ' + base_url + all_key + str(desc_arr[0][0]) + '.csv'
print("For benign : All : " , command)
os.system(command)

for i in range(1, tot_file) :
  file_name = desc_arr[i][1]
  file_rule = desc_arr[i][4]
  file_rule = file_rule.replace('\n', ' ') # to remove newline from filters

  from_loc = base_url + cpac_file_loc + file_name
  to_loc_attack = base_url + attack_key + str(desc_arr[i][0]) + '.csv'
  to_loc_all = base_url + all_key + str(desc_arr[i][0]) + '.csv'

  packet_to_all_command = 'tshark -r {} -T fields -e frame.number -e frame.time_relative -e frame.len -e data.len -e ip.src -e ip.dst -e frame.protocols -E header=y -E separator=, -E quote=d -E occurrence=f > {}'
  packet_to_all_command = packet_to_all_command.format(from_loc, to_loc_all)

  packet_to_attack_command = 'tshark -r {} -T fields -e frame.number -e frame.time_relative -e frame.len -e data.len -e ip.src -e ip.dst -e frame.protocols -E header=y -E separator=, -E quote=d -E occurrence=f \'{}\' > {}'
  packet_to_attack_command = packet_to_attack_command.format(from_loc, file_rule, to_loc_attack)

  print(file_name, " : All : " , packet_to_all_command)
  os.system(packet_to_all_command)
  print(file_name, " : Attack : ", packet_to_attack_command)
  os.system(packet_to_attack_command)
  # break

For benign : All :  tshark -r /content/drive/MyDrive/ML/anomaly/packet/benign-dec.pcap -T fields -e frame.number -e frame.time_relative -e frame.len -e data.len -e ip.src -e ip.dst -e frame.protocols -E header=y -E separator=, -E quote=d -E occurrence=f > /content/drive/MyDrive/ML/anomaly/csv/1.csv
mitm-arpspoofing-1-dec.pcap  : All :  tshark -r /content/drive/MyDrive/ML/anomaly/packet/mitm-arpspoofing-1-dec.pcap -T fields -e frame.number -e frame.time_relative -e frame.len -e data.len -e ip.src -e ip.dst -e frame.protocols -E header=y -E separator=, -E quote=d -E occurrence=f > /content/drive/MyDrive/ML/anomaly/csv/2.csv
mitm-arpspoofing-1-dec.pcap  : Attack :  tshark -r /content/drive/MyDrive/ML/anomaly/packet/mitm-arpspoofing-1-dec.pcap -T fields -e frame.number -e frame.time_relative -e frame.len -e data.len -e ip.src -e ip.dst -e frame.protocols -E header=y -E separator=, -E quote=d -E occurrence=f 'eth.addr == f0:18:98:5e:ff:9f and  ((((ip.src == 192.168.0.16 and ip.dst == 192.16

In [8]:
csv_location = base_url + all_key
attack_location = base_url + attack_key
normal_location = base_url + 'normal/'
print(csv_location, attack_location)

/content/drive/MyDrive/ML/anomaly/csv/ /content/drive/MyDrive/ML/anomaly/attack/


In [9]:
balign = pd.read_csv('/content/drive/MyDrive/ML/anomaly/csv/1.csv')
balign.to_csv('/content/drive/MyDrive/ML/anomaly/normal/1.csv') # movel balign to normal folder

In [11]:
for i in range(1, tot_file) :
  attack_file = attack_location + str(desc_arr[i][0]) + ".csv"
  csv_file = csv_location + str(desc_arr[i][0]) + ".csv"
  normal_file = normal_location + str(desc_arr[i][0]) + ".csv"

  print("Running for : \t", normal_file)
  print("Attack : \t", attack_file)
  print("All : \t\t", csv_file)
  attack = pd.read_csv(attack_file)
  all = pd.read_csv(csv_file)

  print("All shape before dropping records : ", all.shape)

  attack_frames = attack['frame.number']
  index_to_drop = []

  for i in range(len(attack_frames)) :
    frame_num = attack_frames[i]
    index = all[ all['frame.number'] == frame_num ].index[0]
    index_to_drop.append(index)  
  
  all.drop(index_to_drop , inplace=True)
  print("All shape after dropping records : ", all.shape, "\n")
  all.to_csv(normal_file)


Running for : 	 /content/drive/MyDrive/ML/anomaly/normal/2.csv
Attack : 	 /content/drive/MyDrive/ML/anomaly/attack/2.csv
All : 		 /content/drive/MyDrive/ML/anomaly/csv/2.csv
All shape before dropping records :  (65768, 7)
All shape after dropping records :  (30913, 7) 

Running for : 	 /content/drive/MyDrive/ML/anomaly/normal/3.csv
Attack : 	 /content/drive/MyDrive/ML/anomaly/attack/3.csv
All : 		 /content/drive/MyDrive/ML/anomaly/csv/3.csv
All shape before dropping records :  (33121, 7)
All shape after dropping records :  (19987, 7) 

Running for : 	 /content/drive/MyDrive/ML/anomaly/normal/4.csv
Attack : 	 /content/drive/MyDrive/ML/anomaly/attack/4.csv
All : 		 /content/drive/MyDrive/ML/anomaly/csv/4.csv
All shape before dropping records :  (34043, 7)
All shape after dropping records :  (18899, 7) 

Running for : 	 /content/drive/MyDrive/ML/anomaly/normal/5.csv
Attack : 	 /content/drive/MyDrive/ML/anomaly/attack/5.csv
All : 		 /content/drive/MyDrive/ML/anomaly/csv/5.csv
All shape bef

# Final to use files

# Combine attack files

In [48]:
os.chdir(attack_location)
all_attack_filenames = [i for i in glob.glob('*.{}'.format('csv'))]
print("Total attack files to combine : ", len(all_attack_filenames))

#combine all files in the list
combined_attack_csv = pd.concat([pd.read_csv(f) for f in all_attack_filenames ])

# Check for duplicate and remove if found
duplicate = combined_attack_csv[combined_attack_csv.duplicated()]
duplicate.shape # No duplicate therefore no need to remove anything

Total attack files to combine :  58


(0, 7)

In [49]:
combined_attack_csv.head

<bound method NDFrame.head of      frame.number  frame.time_relative  ...        ip.dst       frame.protocols
0            1322            10.289003  ...           NaN     eth:ethertype:arp
1            1329            10.319395  ...  192.168.0.13  eth:ethertype:ip:tcp
2            1330            10.319517  ...  192.168.0.13  eth:ethertype:ip:tcp
3            1331            10.328305  ...  192.168.0.13  eth:ethertype:ip:tcp
4            1332            10.328423  ...  192.168.0.13  eth:ethertype:ip:tcp
..            ...                  ...  ...           ...                   ...
506         35744           212.306664  ...  192.168.0.24  eth:ethertype:ip:tcp
507         35844           215.092722  ...  192.168.0.24  eth:ethertype:ip:tcp
508         35845           215.092852  ...  192.168.0.24  eth:ethertype:ip:tcp
509         35846           215.092909  ...  192.168.0.24  eth:ethertype:ip:tcp
510         35847           215.092927  ...  192.168.0.24  eth:ethertype:ip:tcp

[1229718 

# Combine Normal files

In [50]:
os.chdir(normal_location)
all_normal_filenames = [i for i in glob.glob('*.{}'.format('csv'))]
print("Total files to combine : ", len(all_normal_filenames))
#combine all files in the list
combined_normal_csv = pd.concat([pd.read_csv(f) for f in all_normal_filenames ])
if combined_normal_csv.shape[1] > 7 :
  combined_normal_csv = combined_normal_csv.drop(columns=combined_normal_csv.columns[0])

# Check for duplicate and remove if found
duplicate_normal = combined_normal_csv[combined_normal_csv.duplicated()]
print("Duplicate shape : ", duplicate_normal.shape) # No duplicate therefore no need to remove anything

Total files to combine :  59
Duplicate shape :  (1027006, 7)


In [51]:
print("Size before duplicate delete : ", combined_normal_csv.shape[0])
combined_normal_csv.drop_duplicates(keep='first', inplace=True)
print("Size after duplicate delete : ", combined_normal_csv.shape[0])

Size before duplicate delete :  2520111
Size after duplicate delete :  1493105


In [52]:
combined_normal_csv.head

<bound method NDFrame.head of         frame.number  ...              frame.protocols
0                  1  ...    eth:ethertype:ip:udp:mdns
1                  2  ...  eth:ethertype:ipv6:udp:mdns
2                  3  ...   eth:ethertype:ip:icmp:data
3                  4  ...   eth:ethertype:ip:icmp:data
4                  5  ...    eth:ethertype:ip:udp:mdns
...              ...  ...                          ...
137391        137392  ...         eth:ethertype:ip:tcp
137392        137393  ...    eth:ethertype:ip:tcp:data
137393        137394  ...         eth:ethertype:ip:tcp
137394        137395  ...         eth:ethertype:ip:tcp
137395        137396  ...         eth:ethertype:ip:tcp

[1493105 rows x 7 columns]>

# combine both class and extra label column

In [53]:
combined_attack_csv['label']=1
combined_normal_csv['label']=0
combined_attack_csv.shape, combined_normal_csv.shape

In [57]:
combined_normal_csv.columns

Index(['frame.number', 'frame.time_relative', 'frame.len', 'data.len',
       'ip.src', 'ip.dst', 'frame.protocols', 'label'],
      dtype='object')

In [58]:
combined_attack_csv.columns

Index(['frame.number', 'frame.time_relative', 'frame.len', 'data.len',
       'ip.src', 'ip.dst', 'frame.protocols', 'label'],
      dtype='object')

In [61]:
dataset = combined_normal_csv.append(combined_attack_csv)

<bound method NDFrame.head of      frame.number  frame.time_relative  ...              frame.protocols  label
0               1             0.000000  ...    eth:ethertype:ip:udp:mdns      0
1               2             0.000077  ...  eth:ethertype:ipv6:udp:mdns      0
2               3             0.510334  ...   eth:ethertype:ip:icmp:data      0
3               4             0.510989  ...   eth:ethertype:ip:icmp:data      0
4               5             0.719298  ...    eth:ethertype:ip:udp:mdns      0
..            ...                  ...  ...                          ...    ...
506         35744           212.306664  ...         eth:ethertype:ip:tcp      1
507         35844           215.092722  ...         eth:ethertype:ip:tcp      1
508         35845           215.092852  ...         eth:ethertype:ip:tcp      1
509         35846           215.092909  ...         eth:ethertype:ip:tcp      1
510         35847           215.092927  ...         eth:ethertype:ip:tcp      1

[2722823 

After shuffling

In [62]:
dataset = dataset.sample(frac=1).reset_index(drop=True)
dataset.head

<bound method NDFrame.head of          frame.number  frame.time_relative  ...            frame.protocols  label
0               88211            70.949809  ...       eth:ethertype:ip:tcp      0
1               58708           132.842376  ...       eth:ethertype:ip:tcp      0
2              131894           296.013317  ...       eth:ethertype:ip:tcp      0
3              252744            49.393385  ...  eth:ethertype:ip:udp:data      1
4               35069            66.823389  ...       eth:ethertype:ip:tcp      0
...               ...                  ...  ...                        ...    ...
2722818         91235           152.817776  ...  eth:ethertype:ip:udp:data      1
2722819        137817            59.616331  ...  eth:ethertype:ip:udp:data      1
2722820         18178            59.422203  ...       eth:ethertype:ip:tcp      0
2722821         77956           285.853238  ...   eth:ethertype:ip:tcp:ssl      0
2722822        393991            75.815770  ...  eth:ethertype:ip:ud

In [63]:
dataset.to_csv(base_url + 'dataset.csv')